In [ ]:
!pip install fastapi uvicorn scikit-learn pandas numpy joblib pydantic

In [ ]:
#WITHOUT THE HELP OF CO-AGENTS
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib



def detect_target_column(df: pd.DataFrame):
    if "target" in df.columns:
        print("🎯 Rule 1: Found 'target' column.")
        return "target"


    binary_cols = [col for col in df.columns if df[col].nunique() == 2]
    if binary_cols:
        print(f"🎯 Rule 2: Binary target detected -> {binary_cols[-1]}")
        return binary_cols[-1]

    target = df.nunique().sort_values().index[0]

    return target

def ml_trainer_from_csv(csv_path: str):

    df = pd.read_csv(csv_path)
    print(f"📁 Loaded dataset shape: {df.shape}")

    target_column = detect_target_column(df)
    print(f"🎯 Auto-detected target column: {target_column}")

    X = df.drop(columns=[target_column])
    y = df[target_column]

    X = pd.get_dummies(X, drop_first=True)



    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42
    )

    model = RandomForestClassifier(
        n_estimators=100,
        random_state=42
    )

    print("🚀 Training model...")
    model.fit(X_train, y_train)


    print("📊 Evaluating model...")
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)


    model_path = "trained_model.joblib"
    joblib.dump(model, model_path)

    print("\n✅ Training Complete!")
    print(f"📈 Accuracy: {accuracy * 100:.2f}%")
    print(f"💾 Model saved at: {model_path}")

    return {
        "status": "success",
        "accuracy": round(accuracy, 4),
        "target_column": target_column,
        "model": "RandomForestClassifier",
        "model_path": model_path
    }


In [ ]:
result = ml_trainer_from_csv("/content/heart_disease_data.csv")
print(result)

📁 Loaded dataset shape: (303, 14)
🎯 Rule 1: Found 'target' column.
🎯 Auto-detected target column: target
🤖 Creating default Random Forest model...
🚀 Training model...
📊 Evaluating model...

✅ Training Complete!
📈 Accuracy: 82.42%
💾 Model saved at: trained_model.joblib
{'status': 'success', 'accuracy': 0.8242, 'target_column': 'target', 'model': 'RandomForestClassifier', 'model_path': 'trained_model.joblib'}


In [ ]:
#WITH AGENTS

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib


def detect_target_column(df: pd.DataFrame):

    if "target" in df.columns:
        return "target"

    binary_cols = [
        col for col in df.columns
        if df[col].nunique() == 2
    ]
    if len(binary_cols) == 1:
        return binary_cols[0]
    elif len(binary_cols) > 1:

        return binary_cols[-1]


    unique_count = df.nunique().sort_values()
    return unique_count.index[0]



def ml_trainer_agent(
    df: pd.DataFrame,
    model,
    test_size: float = 0.3,
    random_state: int = 42,
    save_model: bool = True
):
    print(f"📁 Training dataset shape: {df.shape}")

    target_column = detect_target_column(df)
    print(f"🎯 Auto-detected target column: {target_column}")


    X = df.drop(columns=[target_column])
    y = df[target_column]


    X = pd.get_dummies(X, drop_first=True)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )


    print("🤖 Training the model...")
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)


    model_path = None
    if save_model:
        model_name = type(model).__name__.lower()
        model_path = f"trained_model_{model_name}.joblib"
        joblib.dump(model, model_path)
        print(f"💾 Model saved to: {model_path}")

    print("\n✅ Training complete!")
    print(f"📊 Accuracy: {accuracy * 100:.2f}%")

    return {
        "status": "success",
        "accuracy": round(accuracy, 4),
        "model_name": type(model).__name__,
        "target_column": target_column,
        "model_path": model_path
    }


In [ ]:
df = synthetic_data_agent()          # your data generator agent
model = model_selector_agent()       # your model selector agent

result = ml_trainer_agent(df, model)

print(result)


NameError: name 'synthetic_data_agent' is not defined